In [1]:
import matplotlib.pyplot as plt
import numpy as np

# Importing Qiskit
from qiskit import IBMQ, BasicAer, Aer
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute

# Import basic plot tools
from qiskit.tools.visualization import plot_histogram

# Scipy
from scipy.optimize import rosen, differential_evolution

# Files
#from google.colab import files

In [2]:
def f1(x):
    return x ** 2 + 1


def f2(x):
    return x ** 2 + x ** 3


def f3(x):
    return np.sin(x + 1)


def f4(x):
    return np.exp(x) + x


TEACHER = np.random.choice(np.linspace(-1, 1), 10)

In [3]:
def U_in(x, xq, qc):
    qc.rz(np.arccos(x ** 2), xq)
    qc.ry(np.arcsin(x), xq)


def U_out(i, j, xq, qc, theta, n):
    t = (i + j * n) * 3
    qc.rx(theta[t + 0], xq[i])
    qc.rz(theta[t + 1], xq[i])
    qc.rx(theta[t + 2], xq[i])


def Hamelton(n, qc):
    for i in range(n - 1):
        qc.cx(i, i + 1)
    #  for i in range(n - 1, 0, -1):
    #    qc.cx(i, i - 1)


def Bfunk(n, iter, count):
    b = [0] * n
    for k in count.keys():
        for i, l in enumerate(k):
            b[i] += int(l) * count[k]
    for i in range(n):
        b[i] /= iter
    return b


In [4]:
def fi(xi, N, deapth, theta, Iterations, Omega):

    xq = QuantumRegister(N, "x")
    yc = ClassicalRegister(N, "y")
    qc = QuantumCircuit(xq, yc)

    U_in(xi, xq, qc) 
    for j in range(deapth):
        Hamelton(N, qc)
        for k in range(N):
            U_out(k, j, xq, qc, theta, N)

    qc.measure(xq, yc)
    job = execute(qc, backend, shots=Iterations)
    result = job.result()
    count = result.get_counts()
    B = Bfunk(N, Iterations, count)
    Y = np.dot(B, Omega)
    return (Y + 1) / 2 + 1

In [10]:
def Lfunk(Theta):
    s = np.pi /2
    Omega = [1/3, 1/3, 1/3]
    Iterations = 1000 
    deapth = 2   
    N = 3 
    L = 0 #np.dot(Omega, Omega) 
    for t in TEACHER:
        L += (f1(t) - fi(t, N, deapth, theta, Iterations, Omega)) ** 2
    return L


def fi(xi, N, deapth, theta, Iterations, Omega):

    xq = QuantumRegister(N, "x")
    yc = ClassicalRegister(1, "y")
    qc = QuantumCircuit(xq, yc)

    #qc.rz(theta[0], xq[0])
    qc.rx(theta[1], xq[0])

    qc.measure(xq[0], yc)
    job = execute(qc, backend, shots=Iterations)
    result = job.result()
    count = result.get_counts()
    
    return sum([float(k)*count[k] for k in count]) / Iterations

def f1(t):
    return 1/2

def GradLfunk(theta):
    gr = [0] * len(theta)
    s = np.pi /2
    Omega = [1/3, 1/3, 1/3]
    Iterations = 1000 
    deapth = 2   
    N = 3 
    for t in TEACHER:
        for i in range(len(theta)):
            theta[i] += s
            A = fi(t, N, deapth, theta, Iterations, Omega)
            theta[i] -= 2 * s
            B = fi(t, N, deapth, theta, Iterations, Omega)
            theta[i] += s
            C = (A - B)
            gr[i] += (fi(t, N, deapth, theta, Iterations, Omega) - f1(t)) * C / np.sin(s)
            #print(np.array(gr))
    return np.array(gr)

In [7]:
def ftestgrad(x):
    return np.array([2 * (x[0] - 1), 2 * (x[1] + 1)])


In [11]:
backend = BasicAer.get_backend('qasm_simulator')
theta = np.array([0, 1], dtype=np.float64)

etta = .01
for i in range(10):
    grad = GradLfunk(theta)
    theta -= etta * GradLfunk(theta)
    print(grad)
    print(Lfunk(theta))


[-0.006941 -2.281209]
0.6645070000000002
[-0.020645 -2.07782 ]
0.6122759999999999
[-0.012417 -2.104573]
0.632545
[ 0.011925 -2.095218]
0.5059910000000001
[-0.014101 -2.008993]
0.47411499999999995
[ 0.017486 -2.000049]
0.43549600000000005
[-1.783000e-03 -1.938023e+00]
0.47902600000000006
[-0.015733 -1.819863]
0.34223100000000006
[ 0.004582 -1.900762]
0.331957
[ 0.011016 -1.802789]
0.30886899999999995
